In [1]:
import numpy as np
import os
import dv_processing as dv
from datetime import timedelta
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import cv2


import sys
sys.path.append('../')
from src.data.preprocessing import *
from src.data.custom_dataset import *
from src.models.convlstm import ConvLSTM
from src.models.spiking_convlstm import SpikingConvLSTM

from src.utils.plotting import *


In [2]:
train_dataloader, test_dataloader = create_dataloader(
    data_dir='../data/seq_data',
    batch_size=64,
    num_workers=47,
)

Total number of sequences: 555
Number of training sequences: 444
Number of test sequences: 111


/home/khoahd/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Training set - Press: 32613, No Press: 100587
Test set - Press: 8574, No Press: 24726
Training DataLoader:
torch.Size([64, 300, 1, 32, 32]) torch.Size([64, 300])
Test DataLoader:
torch.Size([64, 300, 1, 32, 32]) torch.Size([64, 300])
Number of training batches: 7
Number of test batches: 2


## 1. ConvLSTM

In [4]:
checkpoint_path = '../saved_models/convlstm_accumulator/epoch=49-step=350.ckpt'
model = ConvLSTM.load_from_checkpoint(checkpoint_path, in_channels=1, feature_size=256)

In [ ]:
visualize_top_errors(model, test_dataloader, model_name="ConvLSTM", top_n=10, save_path='convlstm_top_errors.mp4')

[(16, 261), (0, 200), (12, 139), (24, 129), (11, 80), (13, 75), (96, 73), (22, 70), (75, 68), (78, 66), (74, 51), (18, 50), (89, 50), (83, 47), (53, 44), (38, 43), (62, 37), (46, 35), (68, 33), (42, 32), (90, 31), (100, 29), (39, 28), (103, 27), (69, 24), (45, 17), (59, 17), (33, 16), (48, 14), (60, 13), (86, 13), (55, 12), (5, 11), (30, 11), (36, 11), (93, 9), (6, 8), (3, 7), (20, 7), (32, 7), (84, 6), (29, 5), (72, 5), (85, 5), (107, 5), (37, 4), (73, 4), (82, 4), (1, 3), (4, 3), (43, 3), (104, 3), (109, 3), (23, 2), (34, 2), (77, 2), (92, 2), (28, 1), (41, 1), (64, 1), (108, 1), (2, 0), (7, 0), (8, 0), (9, 0), (10, 0), (14, 0), (15, 0), (17, 0), (19, 0), (21, 0), (25, 0), (26, 0), (27, 0), (31, 0), (35, 0), (40, 0), (44, 0), (47, 0), (49, 0), (50, 0), (51, 0), (52, 0), (54, 0), (56, 0), (57, 0), (58, 0), (61, 0), (63, 0), (65, 0), (66, 0), (67, 0), (70, 0), (71, 0), (76, 0), (79, 0), (80, 0), (81, 0), (87, 0), (88, 0), (91, 0), (94, 0), (95, 0), (97, 0), (98, 0), (99, 0), (101, 0), 

## Spiking ConvLSTM

In [52]:
spiking_model = SpikingConvLSTM(beta_init=0.9, feature_size=64, spikegrad="fast_sigmoid", in_channels=1)

spiking_model.conv1.weight = model.cnn.conv_block_1[0].weight
spiking_model.conv1.bias = model.cnn.conv_block_1[0].bias

spiking_model.conv2.weight = model.cnn.conv_block_2[0].weight
spiking_model.conv2.bias = model.cnn.conv_block_2[0].bias

spiking_model.slstm.lstm_cell._parameters['weight_ih'] = model.lstm._parameters['weight_ih_l0']
spiking_model.slstm.lstm_cell._parameters['weight_hh'] = model.lstm._parameters['weight_hh_l0']

spiking_model.slstm.lstm_cell._parameters['bias_ih'] = model.lstm._parameters['bias_ih_l0']
spiking_model.slstm.lstm_cell._parameters['bias_hh'] = model.lstm._parameters['bias_hh_l0']


In [49]:
spiking_model.slstm.lstm_cell._parameters['weight_ih'] = model.lstm._parameters['weight_ih_l0']
spiking_model.slstm.lstm_cell._parameters['weight_hh'] = model.lstm._parameters['weight_hh_l0']

spiking_model.slstm.lstm_cell._parameters['bias_ih'] = model.lstm._parameters['bias_ih_l0']
spiking_model.slstm.lstm_cell._parameters['bias_hh'] = model.lstm._parameters['bias_hh_l0']

torch.nn.parameter.Parameter

In [37]:
spiking_model.conv1.bias


Parameter containing:
tensor([ 0.1577, -0.0461, -0.0613,  0.0092,  0.0830, -0.1278],
       requires_grad=True)

In [50]:
model.lstm._parameters['weight_ih_l0']

Parameter containing:
tensor([[ 0.1390, -0.0353, -0.0415,  ..., -0.0828,  0.0327,  0.0670],
        [ 0.0427, -0.0995,  0.0851,  ...,  0.0002, -0.0246,  0.0752],
        [ 0.0574, -0.0263, -0.0512,  ...,  0.0314, -0.0014, -0.0009],
        ...,
        [-0.0124, -0.0165,  0.0848,  ..., -0.1030, -0.0537,  0.0667],
        [ 0.1017,  0.0469, -0.0597,  ..., -0.0440, -0.0618,  0.1148],
        [ 0.0316,  0.0556, -0.1018,  ...,  0.0824, -0.0672,  0.0598]],
       device='cuda:0', requires_grad=True)